> modeling without tensorflow

# 순서

1. 데이터 불러오기
2. 독립변수와 종속변수 분할하기
3. 파라미터 생성
4. 순전파 연산
5. 역전파 연산
6. 파라미터 갱신
7. 손실함수 정의
8. 평가

# Guide

![model guide](img/abalone_mini_without_tf_guide.png)

- main_execute()
    - tf처럼 hyperparameter를 입력받도록 한다.
- math utils?
    - 실행시킬 때 불러올 모듈/데이터들

# main_execute()

함수의 모든 기능을 `main_execute()` 에서 관리하므로 미리 설계한다. 

- 당장은 동작하지 않지만 버튼 먼저 만들자.

In [2]:
def main_execute(epoch_count = 10, mb_size = 2, report = 2, train_ratio = 0.8):
    load_dataset()
    weight_initial, bias_initial = init_param()
    losses_mean_row, accs_mean_row, final_acc = train_and_test(epoch_count, mb_size, report, train_ratio)
    
    return weight_initial, bias_initial, losses_mean_row, accs_mean_row, final_acc 

`main_execute()` 에서 만든 버튼

1. `load_dataset()`
2. `init_param()`
3. `train_and_test()`

상단의 함수를 생성하자.

# MathUnits.ipynb

필요로 하는 파일 (csv 등) 을 ipynb로 작성한다.
- 별도의 파일로 생성함 : `13_MathUnits.ipynb`

# load_dataset()

In [3]:
def load_dataset():
    # 데이터 불러오기
    path = "dataset/dataset/abalone_mini.csv"
    with open(path) as csvfile:
        csvreader = csv.reader(csvfile)
        next (csvfile)

        rows = []
        for row in csvreader:
            rows.append(row)

    # 자주 쓰는 variable globalization
    global input_cnt, output_cnt, data
    
    # input_cnt, output_cnt 생성
    input_cnt, output_cnt = 10, 1

    # one hot vector
    data = np.zeros([len(rows), input_cnt + output_cnt])    # 행은 rows의 개수, 열은 input_cnt + output_cnt

    for idx, row in enumerate(rows):
        # one hot vector
        if row[0] == 'M': data[idx, 0] = 0
        if row[0] == 'F': data[idx, 1] = 1
        if row[0] == 'I': data[idx, 0] = 2

        # 나머지 복사 
        data[idx, 3:] = row[1:]
        

# load `MathUnits.ipynb`

In [4]:
%run 13_MathUnits.ipynb

In [5]:
load_dataset()

# init_param()
parameter 초기화

- weight, bias를 무작위 값으로 초기화한다.

In [9]:
def init_param():
    # weight, bias list
    weight_initial, bias_initial = [], []

    # set initial value of weight, bias
    weight  = np.random.normal(RND_MEAN, RND_STD, size = [ input_cnt, output_cnt ])     # 입력값과 행렬곱을 수행하기 위해 input_cnt, output_cnt로 size를 설정
    bias    = np.zeros([output_cnt]) # np.zeros를 사용한다. (not normal func)
    print(f"Initial Weight Value : \n{weight}")
    print(f"\nInitial Bias Value : \n{bias}")

    # collect previous weight, bias
    weight_initial.append(weight)
    bias_initial.append(bias)

    return (weight_initial, bias_initial)

왜 np.zeros를 사용하나?

- bias의 초기값은 큰 영향을 미치기 때문에 0으로 맞춘다 : 0으로 맞추는 것도 초기화 방식 중 하나


In [7]:
weight_initial, bias_initial = init_model()

Initial Weight Value : 
[[ 0.03598319]
 [ 0.01456214]
 [-0.05456479]
 [-0.06218915]
 [ 0.0222744 ]
 [ 0.03182617]
 [ 0.0325858 ]
 [-0.06482078]
 [ 0.0276185 ]
 [ 0.02679505]]

Initial Bias Value : 
[0.]


# arrange_data()

- 전체 인덱스 탐색
- 데이터 뒤섞기
    - np.random.shuffle(array_name)
- 학습 및 테스트의 분할 인덱스 탐색
- 미니배치의 수 탐색 

In [10]:
def arrange_data(mb_size, train_ratio):
    # 자주 사용하는 데이터 전역변수화
    global shuffle_map, test_begin_index

    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)     # shuffle_map은 단순히 array를 생성한 값이니 이를 섞어 사용한다.

    mini_batch_step_count = int(data.shape[0] * train_ratio // mb_size)     # 학습 데이터의 개수 // mb_size == 전체 개수의 mini_batch의 개수
    test_begin_index = mini_batch_step_count * mb_size # 전체 학습 데이터의 개수 == 테스트 데이터화 학습 데이터의 경계

    return mini_batch_step_count    # 상대적으로 잘 쓰지 않으므로 전역변수화가 아닌 return 값으로 설정


In [11]:
# 함수 test 
mini_batch_step_count = arrange_data(mb_size=2, train_ratio=0.8)

In [13]:
print(f"mini_batch_step_count : {mini_batch_step_count}")
print(f"shuffle_map : {shuffle_map[:3]}")   # 돌릴 때 마다 매번 바뀐다.
print(f"test_begin_index : {test_begin_index}")

mini_batch_step_count : 4
shuffle_map : [7 2 3]
test_begin_index : 8


# get_test_data()

- 테스트 데이터 분할
- 독립 및 종속 변수 생성

In [14]:
def get_test_data():
    test_data = data[shuffle_map[test_begin_index:]]
    return test_data[:, :-output_cnt], test_data[:, -output_cnt:]     # 독립변수, 종속변수 분리하여 추출 : test_x, test_y`

In [17]:
test_x, test_y = get_test_data()
print(test_x)
print("---" * 30)
print(test_y)

[[0.     0.     0.     0.365  0.295  0.08   0.2555 0.097  0.043  0.1   ]
 [2.     0.     0.     0.33   0.255  0.08   0.205  0.0895 0.0395 0.055 ]]
------------------------------------------------------------------------------------------
[[7.]
 [7.]]


# get_train_data()

![epoch idea](img/abalone_mini_without_tf_epoch.png)

- epoch이 진행될 때 마다 shuffle 된다.

아래처럼 처음부터 8개를 다 가져오면 안된다.  

 $\because$ mini batch 단위로 학습하므로
```
def get_train_data():
    train_data = data[shuffle_map[:test_begin_index]]      
````

$\therefore$ minibatch 인덱스의 특징을 찾는다.

minibatch 단위에 따라 인덱스가 달라지는 특징 발견 (0:2, 2:4, 4:6, 6:8)

In [23]:
# [idea]
# mb_size = 2
# def get_train_data():
    # for idx in range((test_begin_index)):
    #     train_data  = data[shuffle_map[ mb_size * idx : mb_size * (idx+1)]]        # ???????????????? 규칙은 알겠는데 왜 data[shuffle_map~ ]이 됐지?
    # return (train_data[:, :-output_cnt], train_data[:, -output_cnt:])
    # ---------------------- 
# 이 때 range는 (==n) 상위 함수에서 받는다 : 따라서 n을 parameter로 받는다. (mb_size도 마찬가지)

def get_train_data(mb_size, n):
    if n == 0:
        np.random.shuffle(shuffle_map[: test_begin_index])  # 돌아갈 때 마다 학습데이터 부분만 섞어라.
    
    train_data = data[shuffle_map[mb_size*n: mb_size * (n+1)]]
    return (train_data[:, :-output_cnt], train_data[:, -output_cnt:])


In [25]:
train_x, train_y = get_train_data(mb_size = 2, n = 0)

print(train_x)
print("===" * 30)
print(train_y)

[[0.     1.     0.     0.44   0.34   0.1    0.451  0.188  0.087  0.13  ]
 [0.     0.     0.     0.5    0.4    0.13   0.6645 0.258  0.133  0.24  ]]
[[10.]
 [12.]]


# run_train()

- 우선 틀만 생성한다.

In [28]:
def run_train(x, y):
    loss = 0 
    accuracy = 100
    return (loss, accuracy)

In [29]:
run_train(0, 0)

(0, 100)

# run_test()

- 순전파 연산 : 결과 확인
- 평가 진행 : 정확도 측정

In [30]:
def run_test(x,y):
    accuracy = 96
    return accuracy

In [31]:
run_test(0,0)

96

# train_and_test()

- 학습 및 테스트 기능 실행
- 정확도 및 손실값 저장
    - 학습을 얼마나 진행할 것인가? : `main_execute()` 의 `train data ratio` 필요
-

In [ ]:
def train_and_test(epoch_count, mb_size, report, train_ratio):
    mini_batch_step_count = arrange_data(mb_size, train_ratio) 
    text_x, test_y = get_test_data()     # test data에 대한 독립과 종속을 밖으로 반환

    for epoch in range(epoch_count):
        # mini batch가 다 돌아야 1 epoch이므로 반복문 하나 더 추가

        loss_list, accuracy_list = [], []
        for n in range(mini_batch_step_count):
            # 학습 데이터를 불러온다.
            train_x, train_y = get_train_data(mb_size, n)      # n : 1epoch을 돌리기 위해 수행하는 mini batch의 step count
            loss, accuracy = run_train(train_x, train_y)

            # 데이터를 불러올 때에도 minibatch size 만큼 쪼개서 가져오고 있다.
            # loss와 accuracy를 어떻게 한번에 보여주나? (like tensorflow) : 하나의 epoch에 대한 loss, acc?
            # -> mini batch들의 값을 모아서 평균을 낸다. : loss_list, accuracy_list

            loss_list.append(loss)
            accuracy_list.append(accuracy)

            ################################### 0721     
            